# Voting Analysis

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import os
from datetime import datetime

In [39]:
cwd = os.getcwd()
date_format = "%Y-%m-%d"
TORNADO_PATH = os.path.join(cwd, "..", "data/tornadoes/US_tornadoes_2000_2022.csv")
TORNADO_SHAPEFILE = os.path.join(cwd, "..", "data/geographic/US_tornadoes_1950_2022/US_tornadoes_1950_2022.shp")
COUNTIES_PATH = os.path.join(cwd, "..", "data/counties/2018_counties.csv")
COUNTIES_SHAPEFILE = os.path.join(cwd, "..", "data/geographic/cb_2018_us_county_5m/cb_2018_us_county_5m.shp")
INTERSECTION_PATH = os.path.join(cwd, "..", "data/tornado_county_intersections.csv")
PERCENT_PATH = os.path.join(cwd, "..", "data/tornadoes_and_counties_new.csv")
RESULTS_PATH = os.path.join(cwd, "..", "data/results_2000s.csv")

# Naive Comparison

In [3]:
def getCountyVoteSet_FlatCount(data_vote,data_tornado,year):
    output = np.array([[0.0,0]]* len(data_vote))
    
    i = 0
    for row in data_vote.index:
        
        filter1 = data_tornado.loc[data_tornado["GEOID"] == data_vote["fips"][row]]
        filter2 = filter1.loc[filter1["yr"].isin(range(year-4,year+1))]
        output[i,1] = len(filter2)
        output[i,0] = data_vote[str(year)][row]
        # print(i)
        i+=1
    
    return output

def getCountyVoteSet_CostAgg(data_vote,data_tornado,year):
    output = np.array([[0.0,0]]* len(data_vote))
    
    i = 0
    for row in data_vote.index:
        
        filter1 = data_tornado.loc[data_tornado["GEOID"] == data_vote["fips"][row]]
        filter2 = filter1.loc[filter1["yr"].isin(range(year-4,year+1))]
        totalCost = sum(filter2["loss"])
        output[i,1] = totalCost
        output[i,0] = data_vote[str(year)][row]
        # print(i)
        i+=1
    
    return output

def getCountyVoteSet_Casualties(data_vote,data_tornado,year):
    output = np.array([[0.0,0]]* len(data_vote))
    
    i = 0
    for row in data_vote.index:
        
        filter1 = data_tornado.loc[data_tornado["GEOID"] == data_vote["fips"][row]]
        filter2 = filter1.loc[filter1["yr"].isin(range(year-4,year+1))]
        totalCost = sum(filter2["inj"]) + sum(filter2["fat"])
        output[i,1] = totalCost
        output[i,0] = data_vote[str(year)][row]
        # print(i)
        i+=1
    
    return output

    
    


In [12]:
data_tornado = pd.read_csv(INTERSECTION_PATH)
data_voting = pd.read_csv(RESULTS_PATH)

# for year in range(2004, 2021, 4):
#     dist_Dem = data_voting.loc[data_voting[str(year)].notnull()]
#     scatters = getCountyVoteSet_Casualties(dist_Dem,data_tornado,year)
#     print(year)
#     plt.scatter(scatters[:,1],scatters[:,0])
#     plt.title(f"{year} Disaster/Voter Turnout")
#     plt.ylabel("% Democrat Share")
#     plt.xlabel("Injuries and Fatalities")
#     plt.show()
    
    
# for year in range(2004, 2021, 4):
#     dist_Dem = data_voting.loc[data_voting[str(year)].notnull()]
    
#     scatters = getCountyVoteSet_CostAgg(dist_Dem,data_tornado,year)
#     print(year)
#     plt.scatter(scatters[:,1],scatters[:,0])
#     plt.title(f"{year} Tornado/Voter Turnout")
#     plt.ylabel("% Democrat Share")
#     plt.xlabel("Monetary Loss")
#     plt.show()
    
# for year in range(2004, 2021, 4):
#     dist_Dem = data_voting.loc[data_voting[str(year)].notnull()]
#     scatters = getCountyVoteSet_FlatCount(dist_Dem,data_tornado,year)
#     print(year)
#     plt.scatter(scatters[:,1],scatters[:,0])
#     plt.title(f"{year} Tornado/Voter Turnout")
#     plt.ylabel("% Democrat Share")
#     plt.xlabel("Total tornadoes")
#     plt.show()
    



# Synthetic Control

In [55]:


def getTornadoSet_Cas(data_tornado, min_cas):

    filter1 = data_tornado[(data_tornado["fat"] + data_tornado["inj"]) * data_tornado["pct_of_tornado"]/100.0 >= min_cas]
    filter2 = filter1[filter1["yr"]>=2000]
    print(filter2[["om","date","st","fat","inj","GEOID","pct_of_tornado"]])
    
    return filter2



# data_tornado = pd.read_csv(TORNADO_PATH)
data_tornado = pd.read_csv(PERCENT_PATH)
data_voting = pd.read_csv(RESULTS_PATH)
tornadoSet = getTornadoSet_Cas(data_tornado,125)
print(len(tornadoSet))


           om        date  st  fat   inj  GEOID  pct_of_tornado
46328     600  2000-12-16  AL   11   144   1125         100.000
46385     634  2000-02-13  GA   11   175  13205         100.000
49923      47  2003-03-20  GA    6   200  13205          84.751
53533    1253  2005-11-06  KY   24   238  18173          50.309
57921     541  2008-04-28  VA    0   200  51800          67.021
58004     666  2008-05-10  OK   21   350  29145          40.670
63000  314625  2011-04-27  AL   64  1500   1125          50.371
63001  314625  2011-04-27  AL   64  1500   1073          40.887
63002  314625  2011-04-27  AL   64  1500   1063           8.713
63069  305268  2011-04-27  GA   20   335  47011          44.850
63395  296616  2011-05-22  MO  158  1150  29145         100.000
63829  323440  2011-06-01  MA    3   200  25013          80.874
65700  451537  2013-05-20  OK   24   212  40027          75.527
67474  606471  2015-12-26  TX   10   468  48113          75.919
73674  617886  2019-05-27  OH    0   166

In [ ]:
def find_synth(data_vote, data_tornado, county, election_year):
    """
    Find a set of counties that can act as a synthetic control for a county, having similar stats, and being far enough away.
    Omit the year that is being observed
    """
    pass
    